# The following are analysis performed for this Bachelor Thesis

In [1]:
%load_ext autoreload
%autoreload 2

from tabulate import tabulate
from texttable import Texttable
import latextable
import header as head
import json
import os

annotations_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations"
community_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection"
bridges_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges"
figure_path = "/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Figures"

There are 74 rumour source tweet with 0 non-rumour source tweet within the total of 74 rumour tweet.

In [ ]:
# p = head.getPath()
# print(p)
head.count()

In [ ]:
# merging following relationship into a JSON file
head.followRelate()

with open('/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json') as f:
    data = json.load(f)
    followRelate_Ids = list(data.keys())

In [ ]:
# generate dictionary for tweetId and userId conversion
# results are saved in Id-conversions folder
head.gen_idConversion()

There are followers who reacted to the source tweets. 
* reacts from followers are saved in `react_follows`
* reacts from non-followers are saved in `react_not_follows`

In [ ]:
# See whether there are any reactors that follows the person they responded to 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/relateIds.json") as f:
    followingRelate = json.load(f)

head.is_follower_react(followingRelate)

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/with_bridges.json") as file:
    communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    who_follows_whom = json.load(file)

bridges = communities["bridges"]
rumours = communities["rumours"]
non_rumours = communities["non_rumours"]

bridges_follow = {}

for bridge in bridges:
    follows = who_follows_whom[bridge]
    bridges_follow[bridge] = {
        "rumours":[],
        "non_rumours":[],
        "bridges":[]
    }

    for f in follows:
        if f in rumours: (bridges_follow[bridge])["rumours"].append(f)
        elif f in non_rumours: (bridges_follow[bridge])["non_rumours"].append(f)
        elif f in bridges: (bridges_follow[bridge])["bridges"].append(f)

head.writeToJSON(head.getRoot(),"bridges_follows",bridges_follow)


In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweetId_annotations.json") as file:
    tweets_type = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as file:
    tweet_to_user = json.load(file)

tweets_type_by_userId = {
    "Source_Tweets": {},
    "Reply_Tweets": {}
}

for id in tweets_type["Source_Tweets"]:
    user = tweet_to_user[id]
    try: (tweets_type_by_userId["Source_Tweets"])[user] = (tweets_type_by_userId["Source_Tweets"])[user]+1
    except : (tweets_type_by_userId["Source_Tweets"])[user] = 1
    # tweets_type_by_userId["Source_Tweets"].append(tweet_to_user[id])

# tweets_type_by_userId["Source_Tweets"] = list(set(tweets_type_by_userId["Source_Tweets"]))

for id in tweets_type["Reply_Tweets"]:
    user = tweet_to_user[id]
    try: (tweets_type_by_userId["Reply_Tweets"])[user] = (tweets_type_by_userId["Reply_Tweets"])[user]+1
    except : (tweets_type_by_userId["Reply_Tweets"])[user] = 1
    # tweets_type_by_userId["Reply_Tweets"].append(tweet_to_user[id])

# tweets_type_by_userId["Reply_Tweets"] = list(set(tweets_type_by_userId["Reply_Tweets"]))
head.writeToJSON(annotations_path,"tweets_type_by_userId",tweets_type_by_userId)

### Reacting Relationship

In [ ]:
reacted_by = head.who_reacted_by_whom()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

rumours = communities["rumours"]
non_rumours = communities["non_rumours"]
uncategorized = communities["uncategorized"]

cat_who_reacted_by_whom = {}

for id in reacted_by:
    if not(id in rumours or id in non_rumours or id in uncategorized):
        continue
    cat_who_reacted_by_whom[id] = {}
    user = cat_who_reacted_by_whom[id]
    user["rumours"] = list(set(reacted_by[id]).intersection(set(rumours)))
    user["non_rumours"] = list(set(reacted_by[id]).intersection(set(non_rumours)))
    user["uncategorized"] = list(set(reacted_by[id]).intersection(set(uncategorized)))

    # pop keys with no reactants
    if not(user["rumours"] or user["non_rumours"] or user["uncategorized"]):
        cat_who_reacted_by_whom.pop(id)

head.writeToJSON(head.getRoot(),"who_reacted_by_whom",cat_who_reacted_by_whom)

In [ ]:
# Filter out type1 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/react/type1.json") as file:
    react_type1 = json.load(file)
    react_type1 = react_type1.keys()

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_reacted_by_whom.json") as file:
    who_follows_whom = json.load(file)

for key, values in who_follows_whom.items():
    bridges = []
    for id in values["rumours"]:
        if id in react_type1:
            bridges.append(id)
            values["rumours"].remove(id)
    
    for id in values["non_rumours"]:
        if id in react_type1:
                bridges.append(id)
                values["non_rumours"].remove(id)
    
    for id in values["uncategorized"]:
        if id in react_type1:
            bridges.append(id)
            values["uncategorized"].remove(id)
    
    (who_follows_whom[key])["bridges"] = bridges

head.writeToJSON(head.getRoot(),"who_reacted_by_whom",who_follows_whom)


Merge following and reacting relationships into one

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_reacted_by_whom.json") as file:
    reacting_relate = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/user_followed_by.json") as file:
    following_relate = json.load(file)

react_ids = reacting_relate.keys()
follow_id = following_relate.keys()

# Find left over 
duplicates_id = list(set(follow_id).intersection(set(react_ids)))
follow_id = list(set(follow_id).difference(set(react_ids)))

for key, values in reacting_relate.items():
    if not(key in duplicates_id) : continue
    
    values["rumours"] = list(set(values["rumours"]).union(set((following_relate[key])["rumours"])))
    values["non_rumours"] = list(set(values["non_rumours"]).union(set((following_relate[key])["non_rumours"])))
    values["bridges"] = list(set(values["bridges"]).union(set((following_relate[key])["bridges"])))
    values["uncategorized"] = list(set(values["uncategorized"]).union(set((following_relate[key])["uncategorized"])))

for follow in follow_id:
    reacting_relate[follow] = following_relate[follow]

head.writeToJSON(head.getRoot(),"react+following",reacting_relate)

Remove bridge nodes from non-bridge followers

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow+react/communities.json") as file:
    FR_communities = json.load(file)
    bridges = FR_communities["bridges"]

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/react+following.json") as file:
    FR_relations = json.load(file)

for key, values in FR_relations.items():
    for id in values["rumours"]:
        if id in bridges: values["rumours"].remove(id); values["bridges"].append(id)
    for id in values["non_rumours"]:
        if id in bridges: values["non_rumours"].remove(id); values["bridges"].append(id)
    for id in values["uncategorized"]:
        if id in bridges: values["uncategorized"].remove(id); values["bridges"].append(id)

head.writeToJSON(head.getRoot(),"react+following",FR_relations)

### Tabular

In [ ]:
# How many users are identified
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/annotations/tweetId_annotations.json") as file:
    annotations = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/Id-conversions/tweet_to_user.json") as file:
    tweet_to_id = json.load(file)

users = set()
for id, types in annotations.items():
    tweets = types.keys()
    for t in tweets:
        users.add(tweet_to_id[t])

print(len(users))


In [ ]:
# Number of users from each community 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

for id, values in communities.items():
    print(id+": "+str(len(values)))

In [46]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/communities.json") as file:
    communities = json.load(file)

for id, values in communities.items():
    entries = [[' '.join([id,"ids"]),"","","","",""]]
    ids = []

    for v in values:
        ids.append(v)
        if len(ids)==6 : 
            entries.extend([ids])
            ids = []
    
    if len(ids) >0 and len(ids) < 6:
        for i in range(6-len(ids)): ids.append("")
        entries.extend([ids])
    
    head.writeToTxt(head.makePath([figure_path]),' '.join([id,"ids"])+"2",head.makeTable(["c","c","c","c","c","c"],["t","t","t","t","t","t"],["i","i","i","i","i","i"],entries))

#### Follow

In [54]:
# Table for type 1 bridges
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/type1.json") as file:
    type1 = json.load(file)

header = ["id"]+["" for i in range(4)]
entries = [header]
inter_data = []
for id in type1:
    inter_data.append(id)
    if len(inter_data) == len(header): 
        entries.append(inter_data)
        inter_data = []

if len(inter_data) > 0: inter_data.extend(["" for i in range(len(header)-len(inter_data))])
entries.append(inter_data)

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow","type 1"]),"type1_ids",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [49]:
# Table for the number of each type of tweet a bridge tweeted
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/tweets_type.json") as file:
    tweets_type = json.load(file)

header = ["id","\# source tweet","\# reply tweet"]
entries = [header]

for id, types in tweets_type.items():
    inter_data = [id]
    for i, num in types.items():
        inter_data.append(num)
    
    entries.append(inter_data)

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow","type 1"]),"tweets_type",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [ ]:
# follow + follower counts
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "m", "b"])
table.add_rows([["id","#rumour followers","#non-rumour followers"]])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = [id,bridge["rum_count"],bridge["nonrum_count"]]
    data.append(info)

table.add_rows(data)
with open('follower_counts.txt', 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))



In [ ]:
# follow + betweeness + centrality
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/type 1/centrality_2.json") as file:
    bridge_centrality = json.load(file)

table = Texttable()
table.set_cols_align(["c", "c", "c"])
table.set_cols_valign(["t", "t", "t"])
table.set_cols_dtype(["i","e","e"])

data = []
for id in bridge_centrality:
    bridge = bridge_centrality[id]
    info = []
    if bridge["rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["rumours","",""])
        
        for follower in bridge["rumours"]:
            user = (bridge["rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    elif bridge["non_rumours"]:
        info.append([id,"betweeness","centrality"])
        info.append(["non-rumours","",""])

        for follower in bridge["non_rumours"]:
            user = (bridge["non_rumours"])[follower]
            info.append([follower,user["betweeness"],user["closeness"]])
    
    data.extend(info)

table.add_rows(data)
# print(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))
with open(head.makePath([figure_path,"follow","follower_centrality.txt"]), 'w') as f:
    f.write(latextable.draw_latex(table, caption="An example table.", label="table:example_table"))



In [5]:
# number of non-rumours followers (sorted)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow/type 1/type1.json") as file:
    type1  = json.load(file)

nr_followers = {}
for id, values in type1.items():
    nr_followers[id] = len(values["non_rumours"])

nr_followers = dict(sorted(nr_followers.items(), key=lambda item: item[1],reverse=True))

header = ["id","# nodes from non-rumours"]
entries = [header]
entries.extend([[x,y] for x,y in nr_followers.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow","type 1"]),"#non-rumour_nodes",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [9]:
# Bridges' closeness centrality in non-rumour community 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow/closeness/non_rumour_closeness.json") as file:
    type1  = json.load(file)

closeness = type1["bridges"]

closeness = dict(sorted(closeness.items(), key=lambda item: item[1],reverse=True))

header = ["id","closeness centrality"]
entries = [header]
entries.extend([[x,y] for x,y in closeness.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i","e"]

head.writeToTxt(head.makePath([figure_path,"follow","type 1"]),"bridge_nr_closeness",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [11]:
# All reachabel nodes from a bridge
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow/reachable_nr_nodes.json") as file:
    type1  = json.load(file)

reachable = {}
for id, values in type1.items():
    reachable[id] = len(values)

reachable = dict(sorted(reachable.items(), key=lambda item: item[1],reverse=True))

header = ["id","# reachable non-rumour nodes"]
entries = [header]
entries.extend([[x,y] for x,y in reachable.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i","i"]

head.writeToTxt(head.makePath([figure_path,"follow","type 1"]),"reachable_nr_nodes",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

#### Follow+React

In [55]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 1/type1.json") as file:
    type1 = json.load(file)

header = ["id"]+["" for i in range(4)]
entries = [header]
inter_data = []
for id in type1:
    inter_data.append(id)
    if len(inter_data) == len(header): 
        entries.append(inter_data)
        inter_data = []

if len(inter_data) > 0: inter_data.extend(["" for i in range(len(header)-len(inter_data))])
entries.append(inter_data)

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 1"]),"type1_ids",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [5]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 3/type 3.json") as file:
    type3 = json.load(file)

header = ["id"]+["" for i in range(4)]
entries = [header]
inter_data = []
for id in type3:
    inter_data.append(id)
    if len(inter_data) == len(header): 
        entries.append(inter_data)
        inter_data = []

if len(inter_data) > 0: inter_data.extend(["" for i in range(len(header)-len(inter_data))])
entries.append(inter_data)

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 3"]),"type3_ids",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [ ]:
# follower counts 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/centralities.json") as file:
    bridge_centrality = json.load(file)

table = head.get_follower_counts(bridge_centrality,["i","i","i"])
with open(head.makePath([head.getRoot(),"Figures","follow+react","F+R_followers_count.txt"]), 'w') as f:
    f.write(table)

In [51]:
# Table for the number of each type of tweet a bridge tweeted
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 1/tweets_type.json") as file:
    tweets_type = json.load(file)

header = ["id","\# source tweet","\# reply tweet"]
entries = [header]

for id, types in tweets_type.items():
    inter_data = [id]
    for i, num in types.items():
        inter_data.append(num)
    
    entries.append(inter_data)

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 1"]),"tweets_type",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [ ]:
# bridge betweeness
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/centralities.json") as file:
    bridge_centrality = json.load(file)

table = head.get_bridge_betweeness(bridge_centrality,["i","e"])
with open(head.makePath([head.getRoot(),"Figures","follow+react","F+R_bridge_betweeness.txt"]), 'w') as f:
    f.write(table)

In [ ]:
# Followers betweeness and centrality
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/centralities.json") as file:
    bridge_centrality = json.load(file)

table = head.get_followers_centralities(bridge_centrality,["i","e","e"])
with open(head.makePath([head.getRoot(),"Figures","follow+react","F+R_followers_centralities.txt"]), 'w') as f:
    f.write(table)

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/threads/en/charliehebdo/552783667052167168/reactions/552785374507175936.json") as file:
    data = json.load(file)

head.writeToJSON(head.makePath([head.getRoot(),"Figures"]),"example_tweet_structure",data)

In [13]:
# Do any non-rumours follows bridges 
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/community_detection/follow/with_bridges.json") as file:
    F_communities = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who-follows-whom.json") as file:
    follows = json.load(file)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/who_followed_by_whom.json") as file:
    followed_by = json.load(file)

for id in F_communities["non_rumours"]:
    try: f = follows[id]; fb = followed_by[id]
    except: continue

    f = [i for i in f if i in F_communities["bridges"] or i in F_communities["rumours"]]
    fb = [i for i in f if i in F_communities["non_rumours"]]

    if len(f) and len(fb): print(id)





In [6]:
# number of non-rumours followers (sorted)

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/bridges/follow+react/type 1/type1.json") as file:
    type1  = json.load(file)

nr_followers = {}
for id, values in type1.items():
    nr_followers[id] = len(values["non_rumours"])

nr_followers = dict(sorted(nr_followers.items(), key=lambda item: item[1],reverse=True))

header = ["id","# nodes from non-rumours"]
entries = [header]
entries.extend([[x,y] for x,y in nr_followers.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i" for i in range(len(header))]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 1"]),"#non-rumour_nodes",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [10]:
# Bridges' closeness centrality in non-rumour community 

with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/closeness/nonrumours+bridges.json") as file:
    type1  = json.load(file)

closeness = type1["bridges"]

closeness = dict(sorted(closeness.items(), key=lambda item: item[1],reverse=True))

header = ["id","closeness centrality"]
entries = [header]
entries.extend([[x,y] for x,y in closeness.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i","e"]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 1"]),"bridge_nr_closeness",head.makeTable(cols_align,cols_valign,cols_dtype,entries))

In [ ]:
with open("/Users/yu-hung/Downloads/pheme-rumour-scheme-dataset/centrality/follow+react/reachable_nr_nodes.json") as file:
    type1  = json.load(file)

reachable = {}
for id, values in type1.items():
    reachable[id] = len(values)

reachable = dict(sorted(reachable.items(), key=lambda item: item[1],reverse=True))

header = ["id","# reachable non-rumour nodes"]
entries = [header]
entries.extend([[x,y] for x,y in reachable.items()])

cols_align = ["c" for i in range(len(header))]
cols_valign = ["t" for i in range(len(header))]
cols_dtype = ["i","i"]

head.writeToTxt(head.makePath([figure_path,"follow+react","type 1"]),"reachable_nr_nodes",head.makeTable(cols_align,cols_valign,cols_dtype,entries))